# Exceções

Já vimos que, quando detectamos erros ou outras situações excepcionais que impedem a continuidade da execução normal do código, podemos lançar uma exceção usando o comando `raise`.

In [1]:
print('Passa aqui.')
raise Exception('Erro!')
print('Não passa aqui')
print('Nem aqui')

Passa aqui.


Exception: Erro!

Como sabemos, se nada for feito a exceção irá interromper a execução do código com uma mensagem de erro, como no código acima.

## 1. Tratando exceções

Mas existe a possibilidade de detectar que a exceção ocorreu e lidar com ela, sem interromper o programa. Para isso, colocamos o código que pode gerar exceção dentro de um bloco `try`, e seguimos esse bloco `try` com um (ou mais) tratadores de exceção, como abaixo.

In [2]:
try:
    print('Something was done.')
    raise Exception('Error!')
    print('Something was not done.')
except Exception:
    print("It didn't work.")
print("Let's finish")

Something was done.
It didn't work.
Let's finish


Normalmente, o `try` envolve uma quantidade maior de código, onde o lançamento da exceção é apenas uma das possibilidades.

O código abaixo tenta re-executar um *loop* se sua execução foi interrompida por exceção.

In [ ]:
completed = False
first = True
while not completed:
    try:
        for i in range(10):
            if first and i == 4:
                raise Exception('Too tired, sorry!')
            print(i, end=' ')
        print()
        completed = True
    except Exception:
        print('Did not complete yet')
        first = False
    print('Tried once.')
print("Now I'm done.")

## 2. Capturando informações do objeto de exceção

Podemos, além de usar a exceção para interromper a execução, também pegar informação sobre o que aconteceu de errado, capturando o objeto de exeção.

In [ ]:
completed = False
first = True
while not completed:
    try:
        for i in range(10):
            if first and i == 4:
                raise Exception('Too tired, sorry!')
            print(i, end=' ')
        print()
        completed = True
    except Exception as expt:
        print('Did not complete yet')
        print(f'The reason given is: "{expt}"')
        first = False
    print('Tried once.')
print("Now I'm done.")

Também é possível colocar outras informações no objeto de exeção, para fornecer mais contexto sobre o erro ocorrido ao tratador da exceção. Para isso, criamos a nossa própria exceção com os dados necessários.

In [ ]:
class TiredException(Exception):
    def __init__(self, it):
        self.it = it
        super().__init__('Too tired, sorry!')

completed = False
first = True
while not completed:
    try:
        for i in range(10):
            if first and i == 4:
                raise TiredException(i)
            print(i, end=' ')
        print()
        completed = True
    except TiredException as expt:
        print('Did not complete yet')
        print(f'The reason given is: "{expt}"')
        print(f'It happened at iteration {expt.it}')
        first = False
    print('Tried once.')
print("Now I'm done.")

Em geral, fazemos nossas exeções serem derivadas de alguma das exceções base pré-definidas no Python. Veja a documentação de [exceções no site do Python](https://docs.python.org/3/library/exceptions.html) e também o [tutorial oficial sobre exceções](https://docs.python.org/3/tutorial/errors.html).

Veremos outro exemplo mais adiante.

## 3. Código de finalização

Após o bloco `try`, além de blocos para tratar de exeções, podemos também definir um bloco `finally`. Esse bloco deve incluir código que será executado tanto se o `try` terminar normalmente (sem exeções), como se ele for interrompido por exceção, **mesmo que a exceção não seja capturada por um tratador apropriado**.

No código abaixo, note como `'End of the silly function'` é impresso tanto no término normal como no termino por interrupção.

In [ ]:
def silly(interrupt):
    try:
        if interrupt:
            raise ValueError()
            # tente também: raise Exception()
        print('This!', end=' ')
        for i in range(3):
            print('Yeah!', end=' ')
        print()
    except ValueError: # Experimente retirar este tratador
        print("I don't like that value")
    finally:
        print('End of the silly function')

silly(True)
print()
silly(False)
print()
silly(1)
print()
silly(None)

Um outro tipo de bloco que pode ser adicionado após o `try` é o bloco `else`. O código desse bloco será executado apenas se o `try` **não** for interrompido por uma exceção.

In [ ]:
def also_silly(interrupt):
    try:
        if interrupt:
            raise ValueError()
        print('This!', end=' ')
        for i in range(3):
            print('Yeah!', end=' ')
        print()
    except ValueError:
        print("I don't like that value!")
    else:
        print('Now you where efficient.')
    finally:
        print('End of another silly function.')

also_silly(True)
print()
also_silly(False)

## 4. Exceções personalizadas (novamente)

É frequente que desejemos criar novos **tipos de exceção**, para exprimir de forma mais clara os **tipos de erro** do nosso programa.

Para isso, como exemplificado antes, basta definir uma classe derivada de `Exception` ou de outra exceção pré-definida. A classe, em princípio, não precisa de nenhum conteúdo novo (mas veja a seguir situação onde isso pode ser útil). Ela servirá apenas, através de seu tipo, para expressar um tipo específico de erro. (Execute o código abaixo diversas vezes para ver a variação aleatória.)

In [ ]:
class BadMood(Exception):
    pass

In [ ]:
import datetime
import random

In [ ]:
h, m = random.randint(0, 23), random.randint(0, 59)
alarm_rings = datetime.time(h, m)
if alarm_rings < datetime.time(11, 0):
    raise BadMood('That is too early!')
print('Good Morning!')

São comuns situações onde, além da informação do *tipo* do erro, queremos também alguma informação de contexto que permita determinar melhor a razão do erro.

Neste caso, é aconselhavel criar novas classes de exceção que recebam essa informação adicional.

In [ ]:
class InvalidDeposit(Exception):
    def __init__(self, mess, val):
        self._val = val
        super().__init__(mess)
        
    @property
    def value(self):
        return self._val
        
class Account:
    def __init__(self, initial_deposit):
        if initial_deposit < 0:
            raise InvalidDeposit('Negative initial deposit', initial_deposit)
        self._balance = initial_deposit
        
    def deposit(self, value):
        if value < 0:
            raise InvalidDeposit('Negative deposit', value)
        self._balance += value

In [ ]:
c = Account(100)

In [ ]:
try:
    c.deposit(-10)
except InvalidDeposit as dep:
    print('The value', dep.value,'is invalid')

In [ ]:
try:
    c2 = Account(-10)
except InvalidDeposit as dep:
    print(dep.value, 'is not a valid initial deposit')

## 5. Lidando com múltiplos tipos de exceção

Até aqui, temos apenas um tratador de exceção para o bloco `try`, mas isso não é obrigatório. Podemos colocar um tratador de exceção para cada tipo de exceção que pode ocorrer durante a execução do `try`. Quando ocorre uma exceção, o Python irá verificar um por um os tratadores de exceção na ordem apresentada, buscando um que seja *compatível* com a exceção gerada. Para ser compatível, ele precisa tratar exatamente a classe da exceção ou uma de suas classes base. Por exemplo, um tratador para `Exception` irá cuidar de todos os tipos (normais) de exceção. (Execute o código abaixo diversas vezes.)

In [ ]:
c = Account(10)

try:
    if random.random() < 0.3:
        raise ValueError('Wrong value')
    elif random.random() < 0.3:
        raise TypeError('Wrong type')
    else:
        c.deposit(-10)
except InvalidDeposit as dep:
    print('The value', dep.value,'is not valid')
except ValueError:
    print('We had an value error')
except Exception:
    print('We got some exception')

## 6. Ordem de blocos num `try`

A ordem dos blocos após o bloco `try` deve ser:

1. Blocos de tratamento de exceção (do mais específico para o mais geral).
1. Bloco `else`.
1. Bloco `finally`.


## 7. `raise` e `try`

É importante entender o que acontece quando um `raise` é executado: o `raise` provoca o lançamento da exceção associada e a interrupção do código em execução.  A execução será retomada apenas quando for encontrado um bloco `try` envolvendo o ponto onde o `raise` foi executado. Isso implica a interrupção da execução de todas as funções e métodos desde o ponto do `try` até o ponto onde o `raise` foi executado. Quando o `try` é encontrado, os blocos `except` associados são avaliados para ver se algum é compatível com o tipo de exceção lançado. Se um deles for, o bloco except correspondente é executado; se nenhum for, as execuções continuam a ser interrompidas em busca de um novo bloco `try`. Se nenhum bloco `try` com um `except` compatível é encontrado, a execução do código é terminada.

In [ ]:
class FirstException(Exception):
    pass

class SecondException(Exception):
    pass

class ThirdException(Exception):
    pass

def f(interrupt):
    print('Starting f')
    if interrupt == 1:
        raise FirstException()
    elif interrupt == 2:
        raise SecondException()
    elif interrupt == 3:
        raise ThirdException()
    print('Ending f')
    
def g(interrupt):
    try:
        print('Starting g')
        f(interrupt)
        print('Ending g')
    except FirstException:
        print('g got a FirstException')
    finally:
        print('g is finallizing')
        
def h(interrupt):
    try:
        print('Starting h')
        g(interrupt)
        print('Ending h')
    except SecondException:
        print('h got a SecondException')
    else:
        print('h or else!')

In [ ]:
print('Run some code.')
h(1)
print('Back again.')

In [ ]:
print('Run some code.')
h(2)
print('Back again.')

In [ ]:
print('Run some code.')
h(3)
print('Back again.')

## 8. Uso de exceções

Os códigos de demonstração acima dão uma idéia errada sobre como exceções são de fato utilizadas em códigos reais.

A grande vantagem do uso exceções é que separamos duas partes do tratamento de erros:

- O momento em que o erro é detectado.
- O momento onde vamos lidar com o erro.

Em código reais, organizados em funções e métodos, é frequente que no momento em que detectamos um erro não temos informações de contexto suficiente para saber como lidar com o erro. Por exemplo, suponha uma função que lê dados de um arquivo de entrada. Se ocorre um erro numa das leituras, o que deve ser feito? Interromper a execução do programa? Tentar acessar dados de outro arquivo? Fornecer algum tipo de valor _default_? Dificilmente a função responsável pela leitura dos dados vai saber qual a ação apropriada. Isto será conhecido apenas do código que realizou a chamada da função. Isto quer dizer que precisamos separar a detecção do erro do seu tratamento, que é o que conseguimos com exceções.

- Ao detectarmos um problema, realizamos um `raise` de uma exceção do tipo apropriado.
- No código que faz operações que podem dar errado e que sabe como lidar com os possíveis erros, usamos `try ... except` para lidar apropriadamente com os erros.

Vejamos um exemplo simples organizado dessa forma.

In [ ]:
def to_int_list(list_string):
    pieces = list_string.split()
    return [int(piece) for piece in pieces]

first = '1 2 3 4 5 6 7 8 9 10'
second = '10 20 30 40 50 6O 70 80 90 100'

try:
    first_list = to_int_list(first)
    second_list = to_int_list(second)
    print([x + y for x, y in zip(first_list, second_list)])
except ValueError as ve:
    print(f'Formatting error in one of the strings: {ve}.')

Onde está a detecção de erros aqui? Ela está na chamada a `int()` feita dentro da função `to_int_list`. Quando chamamos `int` passando uma cadeia de caracteres, ela tenta ler essa cadeia como o valor de um inteiro. Mas o que acontece que ela não consegue fazer isso, como no caso da tentativa de conversão de `6O` para `int` neste exemplo? Não existe forma pela qual a função `int` possa saber como lidar com esse problema. Então ela lança uma exceção do tipo `ValueError`, que interrompe a sua execução e volta para a função `to_int_list`. Esta função também não sabe como lidar com esse erro, então ela simplesmente repassa a exceção para o local onde ela foi chamada. Nesse ponto, como a chamada está dentro de um bloco `try` o código tenta achar um tratador de exceção apropriado, o que ele consegue, e então executa esse tratador (que no caso apenas imprime uma mensagem).

In [ ]:
int('1e')

## 9. Recomendações

O uso de exceções para lidar com erros e situações excepcionais é conveniente e idiomático em Python. Isso significa que várias funções e métodos das bibliotecas de Python lançam exceções nessas situações.

A forma como o interpretador Python lida com exceções por _default_, isto é, interromper a execução do programa com uma mensagem de erro, raramente é o que se deseja.

Portanto, é importante manter isso em mente ao desenvolver seus códigos, e sempre considerar as possíveis exceções:

- Ao lidar com recursos do sistema (por exemplo, arquivos), ou qualquer tipo de objeto que requeira limpeza quando não mais necessário, você deve sempre usar contextos e blocos `with`. Isso garante que a liberação do recurso ou as operações de limpeza sejam realizadas mesmo que ocorra uma exceção.
- Considere com cuidado em que ponto do código deve ser feito o tratamento de cada tipo de erro ou exceção que pode ocorrer, para inserir os `try/except` adequados.

--- 

Apesar da conveniência, o uso de exceções não é sem problemas. Exemplos de dois problemas são:

- Existe um custo adicional durante a execução, pois cada chamada e retorno de função tem que realizar certas operações adicionais, mesmo que nenhuma exceção tenha sido levantada. Esse problema não é especialmente importante em Python, pois ela não é uma linguagem focada em desempenho de execução. Mas isso impede o uso de exceções em certos sistemas com recursos limitados.
- Existe sempre a possibilidade de que uma exceção seja levantada e não tratada, o que levará ao término do programa, o que não é uma opção em diversas situações! 

Por exemplo, o código  de convenções de programação em C++ do Google proíbe o uso de exceções. A razão provavelmente é uma mistura dos dois problemas citados acima: eles precisam do máximo de desempenho possível (cada perda de desempenho significa que eles precisam comprar mais computadores para dar conta da mesma carga) e também não é viável que um dos serviços deles pare de funcionar pois uma exceção foi levantada e não tratada.

# Exercício

Qual a saída produzida pelo código abaixo:
```python
class Err1(Exception):
    pass

class Err2(ValueError):
    pass

class Err3(Err1):
    pass

def f(n):
    print('f start')
    if n == 1:
        raise Err1('one error')
    elif n == 2:
        raise Err2('other error')
    elif n == 3:
        raise Err3('even other error')
    elif n <= 0:
        raise Exception("Don't know")
    print('f finish')
       
def g():
    print('g start')
    for n in range(1, 5):
        try:
            f(n)
        except Err1:
            print('g got Err1')
        except Err2:
            print('g got Err2')
        except Err3:
            print('g got Err3')
        except Exception:
            print('g got some error')
        else:
            print(f'All ok for g, iteration {n}')
        finally:
            print(f'Iteration {n} done in g')
    print('g finish')
           
def h():
    print('h start')
    for n in range(5):
        try:
            f(n)
        except ValueError:
            print('h got a value error')
        except Exception:
            print('h got some error')
        else:
            print(f'All ok for h, iteration {n}')
        finally:
            print(f'Iteration {n} done in h')
    print('h finish')
           
def m():
    print('m start')
    for n in range(5):
        try:
            f(n)
        except Err2:
            print('m got Err2')
        except Err3:
            print('m got Err3')
        else:
            print(f'All ok for m, iteration {n}')
        finally:
            print(f'Iteration {n} done in m')
    print('m finish')
       
try:
    g()
    h()
    m()
except Exception as e:
    print(f'{e}')
```